In [1]:
filmn = "Aladdin (1992)"
t = filmn if not filmn.endswith(")") else filmn[:-7]
t

'Aladdin'

In [2]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import pandas as pd
import requests
import json
import ast
import pprint
import numpy as np

from datetime import datetime

pd.set_option("display.float_format", lambda x: f"{x :.2f}")
pd.set_option("display.max_columns", None)
from get_dataframes import GetDataframes


from tools import import_config, import_datasets, logging

In [3]:
import requests
import json
import ast
import pprint


api_key = "fe4a6f12753fa6c12b0fc0253b5e667f"
language = "fr-FR"
imdb_id = 53926

params = {
    "api_key": api_key,
    "include_adult": "False",
    "language": language,
    "append_to_response": "keywords,credits,videos"
}

find = False

# if find:
#     params.update(
#         {"external_source" : "imdb_id"}
#     )
#     find_id = "https://api.themoviedb.org/3/find/"
# else:
base_url = "https://api.themoviedb.org/3/movie/"
person_url = "https://api.themoviedb.org/3/person/"

url = f"{base_url}{imdb_id}"

r = requests.get(url, params=params)
data = r.json()

cc = [
    # ("keywords", "keywords", "name", "keywords"),
    ("genres", "genres", "name"),
    ("spoken_languages", "spoken_languages", "iso_639_1"),
    ("production_companies_name", "production_companies", "name"),
    ("production_countries", "production_countries", "iso_3166_1"),
    (
        "production_companies_country",
        "production_companies",
        "origin_country",
    ),
]

for k, c, v, *sb in cc:
    if sb:
        data[k] = [k[v] for k in data[c][sb[0]]]
    else:
        data[k] = [k[v] for k in data[c]]

# data["keywords"] = [n["name"] for n in data["keywords"]["keywords"][:4]]
# data["actors"] = [n["name"] for n in data["credits"]["cast"] if n["known_for_department"] == "Acting" and n["order"] <= 2]
# data["director"] = [n["name"] for n in data["credits"]["crew"] if n["job"] == "Director"]
# data["url"] = f"https://www.imdb.com/title/{data['imdb_id']}"
# data["image"] = f"https://image.tmdb.org/t/p/w300_and_h450_bestv2{data['poster_path']}"
# data["youtube"] = [f"https://www.youtube.com/watch?v={n['key']}" for n in data["videos"]["results"]]

to_pop = (
    "belongs_to_collection",
    "production_companies",
    "credits",
)
for tp in to_pop:
    data.pop(tp)

data["release_date"] = data["release_date"][:4]
pprint.pprint(data)

{'adult': False,
 'backdrop_path': '/cl0ZPqF1Ha4HKVPWJx3VBxMwPT4.jpg',
 'budget': 0,
 'genres': ['Animation', 'Comédie'],
 'homepage': '',
 'id': 53926,
 'imdb_id': 'tt0040260',
 'keywords': {'keywords': [{'id': 6513, 'name': 'cartoon'},
                           {'id': 263548, 'name': 'short film'}]},
 'original_language': 'en',
 'original_title': 'Daddy Duck',
 'overview': 'Voulant adopter un bébé, Donald se retrouve par erreur avec un '
             "bébé kangourou. Le ramenant chez lui, il apprendra que s'occuper "
             "d'un kangourou n'est pas de tout repos...",
 'popularity': 2.741,
 'poster_path': '/sU6JpXhsx2EPHoEdBjWe5Zp6ld7.jpg',
 'production_companies_country': ['US'],
 'production_companies_name': ['Walt Disney Productions'],
 'production_countries': ['US'],
 'release_date': '1948',
 'revenue': 0,
 'runtime': 6,
 'spoken_languages': ['es', 'en'],
 'status': 'Released',
 'tagline': '',
 'title': 'Papa Canard',
 'video': False,
 'videos': {'results': []},
 'vote_ave

In [4]:
import pandas as pd
import ast

df = pd.read_csv("movies_datasets/tmdb_full.csv", low_memory=False)


def decode_clean(item: str) -> str:
    return (
        item.replace("[", "")
        .replace("]", "")
        .replace("'", "")
        .replace('"', "")
    )


def safe_literal_eval(s):
    try:
        return ast.literal_eval(s) if isinstance(s, str) else s
    except:
        return []


col = [
    "genres",
    "spoken_languages",
    "production_companies_name",
    "production_countries",
    "production_companies_country",
]

for c in col:
    df[c] = df[c].apply(
        lambda x: [
            decode_clean(item)
            for item in safe_literal_eval(x)
            if isinstance(item, str) and item.strip()
        ]
        if isinstance(x, str)
        else []
    )

In [7]:
df.columns

Index(['budget', 'titre_genres', 'tmdb_id', 'titre_id', 'overview',
       'popularity', 'production_countries', 'date', 'revenue', 'runtime',
       'tagline', 'titre_str', 'rating_avg', 'rating_vote', 'keywords',
       'production_companies_name', 'actors', 'actors_ids', 'director',
       'director_ids', 'url', 'image', 'youtube', 'titre_clean'],
      dtype='object')

In [87]:
df = pd.read_parquet("streamlit/datasets/site_web.parquet")
import ast
to_clean = [
    "production_countries",
    "production_companies_name",
]
for t in to_clean:
    df[t] = (
        df[t]
        .apply(lambda x: ", ".join(map(str, x)))
        .replace(" ", "")
    )

# clean_act_dct = ["actors_ids", "director_ids"]
# for act_dct in clean_act_dct:
#     df[act_dct] = df[act_dct].apply(lambda x: ast.literal_eval(x))


In [7]:
import pandas as pd
df = pd.read_parquet("streamlit/datasets/site_web.parquet")

In [3]:
def get_actors_dict(df: pd.DataFrame) -> dict:
    actors_dict = {}
    for actors, ids in zip(df.actors, df.actors_ids):
        actors_list = actors.split(", ")
        actor_id_pairs = zip(actors_list, ids)
        actors_dict.update(actor_id_pairs)
    return actors_dict

def get_directors_dict(df: pd.DataFrame) -> dict:
    directors_dict = {}
    for directors, ids in zip(df.director, df.director_ids):
        directors_list = directors.split(", ")
        directors_id_pairs = zip(directors_list, ids)
        directors_dict.update(directors_id_pairs)
    return directors_dict


In [29]:
dfs = df[df["titre_str"] == "Avatar"]

name_actors = [k for k in get_actors_dict(dfs).keys()]
list_acotrs = [k for k in get_actors_dict(dfs).values()]

name_actors, list_acotrs
# il faut que je passe la liste a fetch actors bio et que je récupere les images des films dans

(['Sam Worthington', 'Zoe Saldaña', 'Stephen Lang'], [65731, 8691, 32747])

In [9]:
def fetch_actors_bio(imdb_id: int):
    api_key = "fe4a6f12753fa6c12b0fc0253b5e667f"
    language = "fr-FR"
    params = {
        "api_key": api_key,
        "include_adult": "False",
        "language": language,
        "append_to_response": "combined_credits",
    }
    base_url = "https://api.themoviedb.org/3/person/"
    url_image = "https://image.tmdb.org/t/p/w300_and_h450_bestv2"
    url_youtube = "https://www.youtube.com/watch?v="
    url = f"{base_url}{imdb_id}"
    r = requests.get(url, params=params)
    data = r.json()

    data["image"] = f"{url_image}{data['profile_path']}"
    top_credits = sorted(
        (
            n for n in data["combined_credits"]["cast"]
            if n["media_type"] == "movie" and n["order"] <= 3
            and all(genre not in n["genre_ids"] for genre in [99, 16, 10402]) # 99: Documentaire, 16: Animation, 10402: Musique
        ),
        key=lambda x: (-x['popularity'], -x['vote_average'], -x["vote_count"])
    )[:8]
    data["top_5"] = [n["title"] for n in top_credits]
    data["top_5_images"] = [f"{url_image}{n['poster_path']}" for n in top_credits]
    data["top_5_TMDb_ids"] = [n['id'] for n in top_credits]

    to_pop = (
        "adult",
        "also_known_as",
        "gender",
        "homepage",
        "profile_path",
        "combined_credits",
        "known_for_department",
    )
    for tp in to_pop:
        data.pop(tp)
    return data


fetch_actors_bio(287)


{'biography': "William Bradley Pitt, dit Brad Pitt, est un acteur et producteur de cinéma américain né le 18 décembre 1963 à Shawnee (Oklahoma).\n\nRepéré dans une publicité pour Levi's, Brad Pitt sort de l'anonymat grâce à un petit rôle dans le film Thelma et Louise de Ridley Scott. En très peu de temps, il devient une véritable star et sa collaboration avec le réalisateur David Fincher donne naissance aux films cultes Seven, Fight Club et L'Étrange Histoire de Benjamin Button. Il tourne dans de nombreux autres succès comme Entretien avec un vampire de Neil Jordan, Ocean's Eleven et ses suites de Steven Soderbergh, Troie de Wolfgang Petersen, Inglourious Basterds de Quentin Tarantino et World War Z de Marc Forster. Au cours de sa carrière, il a reçu quatre nominations aux Oscars et cinq nominations aux Golden Globes, dont un remporté pour L'Armée des douze singes de Terry Gilliam en 1996.\n\nSex-symbol des années 1990, Brad Pitt est le premier acteur élu deux fois « Homme le plus sexy

In [1]:
def fetch_directors_bio(imdb_id: int):
    api_key = "fe4a6f12753fa6c12b0fc0253b5e667f"
    language = "fr-FR"
    params = {
        "api_key": api_key,
        "include_adult": "False",
        "language": language,
        "append_to_response": "combined_credits",
    }
    base_url = "https://api.themoviedb.org/3/person/"
    url_image = "https://image.tmdb.org/t/p/w300_and_h450_bestv2"
    url_youtube = "https://www.youtube.com/watch?v="
    url = f"{base_url}{imdb_id}"
    r = requests.get(url, params=params)
    data = r.json()

    data["image"] = f"{url_image}{data['profile_path']}"
    top_credits = sorted(
        (
            n for n in data["combined_credits"]["crew"]
            if n["media_type"] == "movie" and n["job"] =="Director"
            and all(genre not in n["genre_ids"] for genre in [99, 10402]) # 99: Documentaire, 16: Animation, 10402: Musique
        ),
        key=lambda x: (-x['popularity'], -x['vote_average'], -x["vote_count"])
    )[:8]
    data["top_5"] = [n["title"] for n in top_credits]
    data["top_5_images"] = [f"{url_image}{n['poster_path']}" for n in top_credits]
    data["top_5_TMDb_ids"] = [n['id'] for n in top_credits]

    to_pop = (
        "adult",
        "also_known_as",
        "gender",
        "homepage",
        "profile_path",
        "combined_credits",
        "known_for_department",
    )
    for tp in to_pop:
        data.pop(tp)
    return data


fetch_directors_bio(488)

NameError: name 'requests' is not defined

In [1]:
t = {'biography': "Steven Spielberg est un réalisateur, scénariste et producteur de cinéma américain, né le 18 décembre 1946 à Cincinnati (Ohio).\n\nIssu de la deuxième génération du Nouvel Hollywood dans les années 1970, il réalise le premier blockbuster de l'histoire du cinéma : Les Dents de la mer. Il enchaîne les succès (E.T. l'extra-terrestre, série Indiana Jones, Jurassic Park) tout en développant ses activités de gestionnaire. Fondateur de la société de production Amblin et cofondateur du studio DreamWorks SKG, il produit de nombreux films à succès (Poltergeist, Gremlins, Retour vers le futur, Qui veut la peau de Roger Rabbit ou encore la trilogie Men in Black et Transformers). Il a également financé et distribué des œuvres plus exigeantes ou moins grand public telles que Lettres d'Iwo Jima de Clint Eastwood, American Beauty de Sam Mendes et Hollywood Ending de Woody Allen.",
 'birthday': '1946-12-18',
 'deathday': None,
 'id': 488,
 'imdb_id': 'nm0000229',
 'name': 'Steven Spielberg',
 'place_of_birth': 'Cincinnati, Ohio, USA',
 'popularity': 32.328,
 'image': 'https://image.tmdb.org/t/p/w300_and_h450_bestv2/tZxcg19YQ3e8fJ0pOs7hjlnmmr6.jpg',
 'top_5': ['Indiana Jones et la dernière croisade',
  'La Liste de Schindler',
  'Ready Player One',
  'Arrête-moi si tu peux',
  'Il faut sauver le soldat Ryan',
  'Indiana Jones et le royaume du crâne de cristal',
  'Les Dents de la mer',
  "Les Aventuriers de l'arche perdue"],
 'top_5_images': ['https://image.tmdb.org/t/p/w300_and_h450_bestv2/73gYk4vShggfajlthEcOsY5ZhQI.jpg',
  'https://image.tmdb.org/t/p/w300_and_h450_bestv2/vHgf8NE7tXV4DJPEnqVLZDof8fT.jpg',
  'https://image.tmdb.org/t/p/w300_and_h450_bestv2/832ngDVXhOVp6HBnuMMeeWTrGXN.jpg',
  'https://image.tmdb.org/t/p/w300_and_h450_bestv2/kBKinpkg20wdxjRYtzrJDJfytms.jpg',
  'https://image.tmdb.org/t/p/w300_and_h450_bestv2/uLbBoC6QzfKibKf7Y5u78DR62Rz.jpg',
  'https://image.tmdb.org/t/p/w300_and_h450_bestv2/hYv1vaDEdU9togO6mfEoKnSb3sk.jpg',
  'https://image.tmdb.org/t/p/w300_and_h450_bestv2/wuRf6gkX0JUJsu1MeECPubhMRdd.jpg',
  'https://image.tmdb.org/t/p/w300_and_h450_bestv2/ySr6a1Za5bt00hzPHTiKl4xbajr.jpg'],
 'top_5_TMDb_ids': [89, 424, 333339, 640, 857, 217, 578, 85]}


In [5]:
for k, v in t.items():
    

https://image.tmdb.org/t/p/w300_and_h450_bestv2/tZxcg19YQ3e8fJ0pOs7hjlnmmr6.jpg
https://image.tmdb.org/t/p/w300_and_h450_bestv2/tZxcg19YQ3e8fJ0pOs7hjlnmmr6.jpg
https://image.tmdb.org/t/p/w300_and_h450_bestv2/tZxcg19YQ3e8fJ0pOs7hjlnmmr6.jpg
https://image.tmdb.org/t/p/w300_and_h450_bestv2/tZxcg19YQ3e8fJ0pOs7hjlnmmr6.jpg
https://image.tmdb.org/t/p/w300_and_h450_bestv2/tZxcg19YQ3e8fJ0pOs7hjlnmmr6.jpg
https://image.tmdb.org/t/p/w300_and_h450_bestv2/tZxcg19YQ3e8fJ0pOs7hjlnmmr6.jpg
https://image.tmdb.org/t/p/w300_and_h450_bestv2/tZxcg19YQ3e8fJ0pOs7hjlnmmr6.jpg
https://image.tmdb.org/t/p/w300_and_h450_bestv2/tZxcg19YQ3e8fJ0pOs7hjlnmmr6.jpg
https://image.tmdb.org/t/p/w300_and_h450_bestv2/tZxcg19YQ3e8fJ0pOs7hjlnmmr6.jpg
https://image.tmdb.org/t/p/w300_and_h450_bestv2/tZxcg19YQ3e8fJ0pOs7hjlnmmr6.jpg
https://image.tmdb.org/t/p/w300_and_h450_bestv2/tZxcg19YQ3e8fJ0pOs7hjlnmmr6.jpg
https://image.tmdb.org/t/p/w300_and_h450_bestv2/tZxcg19YQ3e8fJ0pOs7hjlnmmr6.jpg


In [6]:

# if find:
#     params.update(
#         {"external_source" : "imdb_id"}
#     )
#     find_id = "https://api.themoviedb.org/3/find/"
# else:


api_key = "fe4a6f12753fa6c12b0fc0253b5e667f"
language = "fr-FR"
# imdb_id = 120
imdb_id = 2037

params = {
    "api_key": api_key,
    "include_adult": "False",
    "language": language,
    "append_to_response": "combined_credits",
}

find = False

base_url = "https://api.themoviedb.org/3/person/"
url_image = "https://image.tmdb.org/t/p/w300_and_h450_bestv2"
url_youtube = "https://www.youtube.com/watch?v="
# url = f"{base_url}{imdb_id}/movie_credits"
url = f"{base_url}{imdb_id}"

r = requests.get(url, params=params)
data = r.json()

data["image"] = f"{url_image}{data['profile_path']}"
top_credits = sorted(
    (
        n for n in data["combined_credits"]["cast"]
        if n["media_type"] == "movie" and n["order"] <= 3
        and all(genre not in n["genre_ids"] for genre in [99, 16, 10402]) # 99: Documentaire, 16: Animation, 10402: Musique
    ),
    key=lambda x: (-x['popularity'], -x['vote_average'], -x["vote_count"])
)[:8]
data["top_5"] = [n["title"] for n in top_credits]
data["top_5_images"] = [f"{url_image}{n['poster_path']}" for n in top_credits]
data["top_5_TMDb_ids"] = [n['id'] for n in top_credits]

to_pop = (
    "adult",
    "also_known_as",
    "gender",
    "homepage",
    "profile_path",
    "combined_credits",
    "known_for_department",
    # "imdb_id",
    # "popularity",
)
for tp in to_pop:
    data.pop(tp)

# data["release_date"] = data["release_date"][:4]
pprint.pprint(data)
# pprint.pprint(top_credits)
# Supposons que `data` est votre liste de dictionnaires contenant les données des films


{'biography': 'Cillian Murphy est un acteur irlandais de cinéma et de théâtre. '
              'Il est souvent noté par les critiques pour ses performances '
              'caméléoniques dans divers rôles et yeux bleus distinctifs.\n'
              '\n'
              'Originaire de Cork, Murphy a commencé sa carrière en tant que '
              'musicien de rock. Après avoir refusé un contrat '
              "d'enregistrement, il fait ses débuts professionnels dans la "
              'pièce Disco Pigs en 1996. Il joue ensuite dans des films et des '
              'productions théâtrales irlandaises et britanniques à la fin des '
              'années 1990 et au début des années 2000. le héros du film '
              'post-apocalyptique 28 Days Later. Les rôles les plus connus de '
              'Murphy sont les méchants dans deux superproductions de 2005: '
              "l'épouvantail dans le film de super-héros Batman Begins, et "
              'Jackson Rippner dans le thriller Red E

In [7]:
async def get_movie_details(
    ss,
    TMdb_id: int,
    api_key: str,
    language: str,
):
    params = {
        "api_key": api_key,
        "include_adult": "False",
        "language": language,
        "append_to_response": "keywords,credits,videos",
    }

    base_url = "https://api.themoviedb.org/3/movie/"
    url = f"{base_url}{TMdb_id}"

    async with ss.get(url, params=params) as rsp:
        data = rsp.json()
        return await data


In [4]:
import requests
import json
import ast
import pprint
import pandas as pd

api_key = "fe4a6f12753fa6c12b0fc0253b5e667f"
language = "en-FR"
imdb_id = 872585

params = {
    "api_key": api_key,
    "include_adult": "False",
    "language": language,
    "append_to_response": "keywords,credits,videos",
}

base_url = "https://api.themoviedb.org/3/movie/"

url = f"{base_url}{imdb_id}"

r = requests.get(url, params=params)
data = r.json()

cleaning = (
    ("keywords", "keywords", "name", "keywords"),
    ("genres", "genres", "name"),
    ("spoken_languages", "spoken_languages", "iso_639_1"),
    ("production_companies_name", "production_companies", "name"),
    ("production_countries", "production_countries", "iso_3166_1"),
)
for key, category, value, *subkey in cleaning:
    if subkey:
        data[key] = [k[value] for k in data[category][subkey[0]]]
    else:
        data[key] = [k[value] for k in data[category]]
to_pop = [
    "videos",
    # "video",
    "credits",
    "homepage",
    "belongs_to_collection",
    "adult",
    "original_language",
    "backdrop_path",
    "spoken_languages",
    "status",
    "original_title",
    "production_companies",
    "poster_path",
]
for tp in to_pop:
    data.pop(tp)
pprint.pprint(data)

{'budget': 100000000,
 'genres': ['Drama', 'History'],
 'id': 872585,
 'imdb_id': 'tt15398776',
 'keywords': ['husband wife relationship',
              'based on novel or book',
              'politics',
              'atomic bomb',
              'professor',
              'patriotism',
              'new mexico',
              'world war ii',
              'hallucination',
              'atomic bomb test',
              'surrealism',
              'physics',
              'biography',
              'berkeley',
              'based on true story',
              'physicist',
              'jewish american',
              'interrogation',
              'guilt',
              'nonlinear timeline',
              'historical event',
              'nuclear weapons',
              'communism',
              'red scare',
              'prometheus',
              'mccarthyism',
              'top secret project',
              'moral dilemma',
              'usa politics',
              '1940s

In [9]:
cleaning = (
    ("name", "keywords", "keywords"),
    ("name", "genres"),
    ("iso_639_1", "spoken_languages"),
    ("name", "production_companies"),
    ("iso_3166_1", "production_countries"),
)

for key, category, *subkey in cleaning:
    if subkey:
        data[category] = [item[key] for item in data[category][subkey[0]]]
    else:
        data[category] = [item[key] for item in data[category]]

TypeError: list indices must be integers or slices, not str

In [ ]:
def clean_fake_list(s):
    t = []
    for i in ast.literal_eval(s):
        print(i)


df["keywords"] = df["keywords"].apply(clean_fake_list)

In [ ]:
tmd = import_datasets("clean_datasets/tmdb_full.parquet", "parquet")

In [ ]:
tmd

In [ ]:
from datetime import datetime
from dateutil import parser
from time import time


now = datetime.now()


def date_to_ts(d):
    return int(
        parser.parse(d).replace(tzinfo=datetime.timezone.utc).timestamp()
        * 1000
    )


def utc_ms() -> float:
    return datetime.utcnow().timestamp() * 1000


now = str(datetime.now())


def date_to_ts_now():
    return int(datetime.timestamp(datetime.now()))


now

In [ ]:
config = import_config()
datas = GetDataframes(config)
link = "movies_cleaned"
df = datas.get_dataframes(link, True)

In [ ]:
df[df["titre_id"].str.contains("tt5013056")]

In [ ]:
synthese_docstrings = (
    "Compréhension de la Méthode : Comprendre la méthode, ses paramètres, "
    + "processus internes, et sorties. "
    + "Utilisation du Format Numpy : Utiliser le format de docstring Numpy pour "
    + "sa clarté dans la description des paramètres et des valeurs de retour. "
    + "Structure du Docstring : Inclure : "
    + "- Brève Description : Résumer la fonction de la méthode en une ou deux lignes. "
    + "- Parameters : Lister et décrire tous les paramètres. "
    + "- Returns : Expliquer les valeurs retournées par la méthode. "
    + "Rédaction Précise et Limitée : Être concis et précis, respecter la limite "
    + "de 75 caractères (espace compris) par ligne selon PEP8. "
    + "Terminologie Technique Adéquate : Utiliser un langage technique approprié, "
    + "par exemple des termes spécifiques aux DataFrames pour les méthodes les concernant. "
)
print(synthese_docstrings)

In [ ]:
t = "Je suis une IA spécialisée en Python scientifique, avec une expertise dans la rédaction de code et de documentation en français. Mon approche consiste à analyser et comprendre le code en profondeur avant de rédiger des docstrings. Ces dernières sont créées dans le style numpy, visant à être concises, claires, et faciles à comprendre. Je respecte scrupuleusement les règles du PEP8, notamment la limite de 65 caractères par ligne pour garantir une excellente lisibilité. Mon objectif est de fournir des explications détaillées non seulement sur le fonctionnement des fonctions, mais aussi sur leur rôle et leur intégration dans un script, assurant ainsi une compréhension complète pour les utilisateurs."
t = t.split(".")
t
p = (
    "Je suis une IA spécialisée en Python scientifique, avec une expertise dans la rédaction de code et de documentation en français"
    + "Mon approche consiste à analyser et comprendre le code en profondeur avant de rédiger des docstrings"
    + "Ces dernières sont créées dans le style numpy en anglais c'est a dire (Methods, Atributs, Parameters, Returns, Raises, Notes etc...), visant à être concises, claires, et faciles à comprendre"
    + "Je respecte scrupuleusement les règles du PEP8, notamment la limite de 65 caractères par ligne pour garantir une excellente lisibilité"
    + "Mon objectif est de fournir des explications détaillées non seulement sur le fonctionnement des fonctions"
    + "mais aussi sur leur rôle et leur intégration dans un script, assurant ainsi une compréhension complète pour les utilisateurs"
)

In [ ]:
ai_description = (
    "La documentation de la fonction doit débuter par une explication concise et "
    "directe de son but principal, évitant l'usage d'un titre 'description'. Ensuite, "
    "chaque paramètre est décrit individuellement, mentionnant son type, "
    "son rôle et toute contrainte ou valeur par défaut pertinente. Si la fonction "
    "renvoie quelque chose, la nature de cette valeur de retour est également "
    "détaillée, en précisant son type et ce qu'elle représente dans le contexte de "
    "la fonction. Des sections supplémentaires peuvent être ajoutées pour évoquer "
    "des exceptions potentielles levées par la fonction et des notes supplémentaires, "
    "si nécessaire. Cependant, il est important de garder ces sections courtes et "
    "pertinentes. Tout au long de la documentation, le style d'écriture doit être clair, "
    "concis et conforme aux normes PEP8, en veillant à ce que chaque ligne ne dépasse "
    "pas 70 caractères. Cela facilite la lisibilité et la maintenance du code. Le formatage "
    "Black, connu pour son style épuré et sa cohérence, doit être appliqué pour assurer "
    "une présentation uniforme et professionnelle du document."
)

In [ ]:
name = "machine_learning"

logging.info(f"Creating {name} dataframe...")
tmdb_l = "clean_datasets/tmdb_updated.parquet"
actors_l = "clean_datasets/actors_movies.parquet"
directors_l = "clean_datasets/directors_movies.parquet"
movies_l = "clean_datasets/movies_cleaned.parquet"

tmdb = import_datasets(tmdb_l, "parquet")
actors = import_datasets(actors_l, "parquet")
directors = import_datasets(directors_l, "parquet")
movies = import_datasets(movies_l, "parquet")

col_to_keep = [
    "titre_id",
    "titre_str",
    "titre_genres",
    "rating_avg",
    "rating_votes",
]
movies = movies[col_to_keep]

In [ ]:
directors_list_id = directors["titre_id"]
condi = movies["titre_id"].isin(directors_list_id)
condi2 = actors["titre_id"].isin(directors_list_id)
movies = movies[condi]
actors = actors[condi2]

actors_list_id = actors["titre_id"]
condi = movies["titre_id"].isin(actors_list_id)
condi2 = directors["titre_id"].isin(actors_list_id)
movies = movies[condi]
directors = directors[condi2]

In [ ]:
print(len(directors_list_id))
print(len(actors_list_id))
print(len(movies))

In [ ]:
col_to_keep = ["imdb_id", "overview"]
tmdb = tmdb[col_to_keep]

col_to_keep = [
    "titre_id",
    "person_name",
    # "person_index"
]
actors = actors[col_to_keep]

col_to_keep = [
    "titre_id",
    "person_name",
    # "person_index"
]
directors = directors[col_to_keep]

actors.loc[:, "person_name"] = actors["person_name"].str.split(", ")
directors.loc[:, "person_name"] = directors["person_name"].str.split(", ")

person_name = actors.groupby("titre_id")["person_name"].sum().reset_index()
person_list = person_name["person_name"].to_list()

directors_name = (
    directors.groupby("titre_id")["person_name"].sum().reset_index()
)
directors_list = directors_name["person_name"].to_list()

movies["actors"] = person_list
movies["directors"] = directors_list

In [ ]:
print(len(directors_list_id))
print(len(actors_list_id))
print(len(movies))

In [ ]:
logging.info(f"Merging {name} dataframe...")
ml_df = pd.merge(movies, tmdb, left_on="titre_id", right_on="imdb_id")

logging.info(f"Droping NaN {name} dataframe...")
ml_df.drop(["imdb_id"], axis=1, inplace=True)
ml_df[ml_df.isna().any(axis=1)]
ml_df.dropna(inplace=True)

# tt = (
#     ("actors", "actors"),
#     ("titre_genres", "titre_genres"),
#     ("directors", "directors"),
# )
# for t in tt:
#     ml_df[t[0]] = ml_df[t[1]].apply(
#         lambda x: ", ".join(map(str, x))
#     ).replace(" ", "")

# # Full loWer pour reduire les titres, actors, directors etc...
# # for t in tt:
# #     ml_df[t[0]] = ml_df[t[1]].apply(full_lower)
# logging.info(f"Process Overview...")
# ml_df['overview'] = ml_df['overview'].astype(str).apply(clean_overview)

# logging.info(f"Writing {name} dataframe...")
# ml_df.to_parquet(path_file)